# OpenAI Function Calling
Brief example of how to use function calling to extract different types of data

In [10]:
import os
import json
import openai
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG_ID")

In [22]:
def openai_response(messages, functions):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",
    )
    return response["choices"][0]["message"]["function_call"]["arguments"]

In [23]:
document = """OpenAI is an American artificial intelligence (AI) research laboratory consisting of the non-profit OpenAI, Inc. and its for-profit subsidiary corporation OpenAI, L.P.. OpenAI conducts research on artificial intelligence with the declared intention of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work".

OpenAI was founded in 2015 by Ilya Sutskever, Greg Brockman, Trevor Blackwell, Vicki Cheung, Andrej Karpathy, Durk Kingma, Jessica Livingston, John Schulman, Pamela Vagata, and Wojciech Zaremba, with Sam Altman and Elon Musk serving as the initial board members. Microsoft provided OpenAI LP with a $1 billion investment in 2019 and a $10 billion investment in 2023.
"""

### Example structure of using function call for
- objects
- nested objects
- arrays
- enums
- strings
- ints
- numbers
- booleans

In [29]:
messages = [
    {"role": "system", "content": "You are a helpful assistant the parses information into JSON format."},
    {"role": "user", "content": document},
]
functions = [
        {
            "name": "parse_wikipedia_page",
            "description": "Parses a Wikipedia page into JSON format.",
            "parameters": {
                "type": "object",
                "properties": {
                    "people": {
                        "type": "array", 
                        "description": "List of people mentioned in the wikipedia page",
                        "items": {"type": "string", "description": "A person"},
                    },
                    "number_of_people": {
                        "type": "integer",
                        "description": "Number of people mentioned in the wikipedia page",
                    },
                    "summary": {
                        "type": "string",
                        "description": "A summary of the wikipedia page",
                    },
                    "sentiment": {"type": "string", "enum": ["positive", "negative"]},
                    "important_facts": {
                        "type": "object",
                        "description": "Important facts about the wikipedia page",
                        "properties": {
                            "locations": {
                                "type": "array",
                                "description": "Array of locations mentioned in the wikipedia page",
                                "items": {"type": "string", "description": "A location"},
                            },
                            "products": {
                                "type": "array",
                                "description": "Array of products mentioned in the wikipedia page",
                                "items": {"type": "string", "description": "A product"},
                            },
                            "total_investment": {
                                "type": "number",
                                "description": "Sum of the total investment mentioned in the wikipedia page",
                            },
                            "public_company": {
                                "type": "boolean",
                                "description": "Whether the company is publicly traded or not",
                            },
                        },
                    }
                },
                "required": ["people", "number_of_people", "summary", "sentiment", "important_facts"],
            },
        }
    ]

In [30]:
response = openai_response(messages, functions)

In [31]:
print(response)

{
  "people": [
    "Ilya Sutskever",
    "Greg Brockman",
    "Trevor Blackwell",
    "Vicki Cheung",
    "Andrej Karpathy",
    "Durk Kingma",
    "Jessica Livingston",
    "John Schulman",
    "Pamela Vagata",
    "Wojciech Zaremba",
    "Sam Altman",
    "Elon Musk"
  ],
  "number_of_people": 12,
  "summary": "OpenAI is an American artificial intelligence (AI) research laboratory focused on developing safe and beneficial artificial general intelligence. It was founded in 2015 and has received significant investments from Microsoft.",
  "sentiment": "positive",
  "important_facts": {
    "locations": [
      "United States"
    ],
    "products": [
      "artificial general intelligence"
    ],
    "total_investment": 11000000000,
    "public_company": false
  }
}
